# Gradient Boost

In [7]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold, train_test_split
from scipy.stats import randint
from sklearn.metrics import accuracy_score
import joblib

import model_pipeline

seed = 777
rng = np.random.default_rng(seed)

def rng_int():
    return rng.integers(1, 10000)

In [2]:
df = pd.read_csv('../data/preprocessed/_90_drp_outl.csv', index_col='obj_ID')

X = df[['alpha', 'delta', 'u', 'g', 'r', 'i', 'z', 'redshift']]
y = df['class']

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8, stratify=y, random_state=rng_int())

kfold = KFold(n_splits=5, shuffle=True, random_state=rng_int())

pipeline = Pipeline([
    ('classifier', GradientBoostingClassifier()) # Classificador
])

# Definir a distribuição de parâmetros para random search
param_dist = {
    'classifier__n_estimators': randint(50, 400),
    'classifier__learning_rate': [0.01, 0.05, 0.1, 0.2, 0.5],
    'classifier__max_depth': [3, 4, 5, 6],
    'classifier__subsample': [0.8, 0.9, 1.0],
    'classifier__max_features': ['sqrt', 'log2'],
    'classifier__min_samples_split': [10, 20, 50, 100],
    'classifier__min_samples_leaf': [5, 10, 20, 50], 
}

best_model, best_params, best_score = model_pipeline.get_best_params(pipeline, param_dist, kfold, X_train, y_train)

print("Melhores parâmetros:", best_params)
print("Melhor acurácia:", best_score)

Melhores parâmetros: {'classifier__learning_rate': 0.05, 'classifier__max_depth': 6, 'classifier__max_features': 'log2', 'classifier__min_samples_leaf': 50, 'classifier__min_samples_split': 20, 'classifier__n_estimators': 296, 'classifier__subsample': 1.0}
Melhor acurácia: 0.9775743718214311


In [4]:
model_pipeline.plot_learning_curve(best_model, X_train, y_train, cv=kfold, title="Curva de Aprendizado com RandomizedSearchCV")

In [9]:
model_pipeline.dump_model(best_model, 'dump/gradient_boost', X_test, y_test)

In [5]:
from win10toast import ToastNotifier

def notificar(titulo, mensagem):
    toaster = ToastNotifier()
    toaster.show_toast(titulo, mensagem, duration=10)

notificar("Tarefa Concluída", "Seu código terminou de rodar!")

WNDPROC return value cannot be converted to LRESULT


TypeError: WPARAM is simple, so must be an int object (got NoneType)

In [6]:

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

plt.figure(figsize=(8, 6))
sns.heatmap(cm_percent, annot=True, fmt='.1f', cmap='summer')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix Heatmap')
plt.show()